# Byte #3: Function Calling with OpenAI (Agents in Action) 🔧

**⏱️ Time to Complete: 10-15 minutes**

Welcome back to hero HQ! By now you can talk to models, reason through async calls, and keep conversations alive. In this byte you will teach an assistant to reach beyond the model and use trusted tools—perfect for building production-ready copilots that can look up data, run calculations, or trigger workflows on demand.

## Why Function Calling?

Models don’t automatically know your private knowledge base, inventory numbers, or the latest sensor readings. Function calling lets you expose safe, well-defined functions so an assistant can fetch facts, perform actions, or orchestrate other systems—all while you stay in control of what can be executed.

### Core Concepts

- **Tool** → A function you register with the model (JSON schema describes inputs).
- **Tool call** → The model requests a tool by name with arguments when it needs external help.
- **Tool result** → Your application executes the function, then replies with the output so the model can finish its answer.
- **Guardrails** → You decide which tools exist and validate every call before running it.

## Setup Checklist

1. Activate your project virtual environment.
2. Install dependencies once with `uv pip install -e .` so the OpenAI SDK and dotenv helper are ready.
3. Confirm your `.env` file has a valid `OPENAI_API_KEY` (and restart the kernel after edits).

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import json

# Load environment variables so the OpenAI SDK can find OPENAI_API_KEY
load_dotenv()
client = OpenAI()
print("✅ OpenAI client ready for tool calling")

## Step 1: Define a Simple Tool

To keep the flow focused on the Responses API mechanics, we'll expose one playful function: a horoscope generator that always returns a friendly prediction for the requested zodiac sign.

In [ ]:
def get_horoscope(sign: str) -> dict:
    """Return a whimsical horoscope for the requested astrological sign."""
    message = f"{sign}: Next Tuesday you will befriend a baby otter."
    return {"sign": sign, "horoscope": message}

AVAILABLE_TOOLS = {"get_horoscope": get_horoscope}
print("🛠️ Local tools registered: " + ", ".join(AVAILABLE_TOOLS.keys()))

## Step 2: Describe the Tool for the API

The model needs a contract for each function. We'll provide a concise JSON Schema so the assistant knows it can call `get_horoscope` with a single `sign` string.

In [ ]:
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius.",
                },
            },
            "required": ["sign"],
        },
    },
 ]
print(json.dumps(tools, indent=2))

## Step 3: Call the Model with the Horoscope Tool

We will ask the assistant for an Aquarius horoscope. The model may call our tool, so we'll capture that request, execute the Python function, and return the result so it can finish the answer.

In [ ]:
question = "What is my horoscope? I am an Aquarius."

# Create a running input list we will add to over time
input_list = [
    {
        "role": "user",
        "content": [
            {"type": "input_text", "text": question}
        ],
    }
 ]

response = client.responses.create(
    model="gpt-5-nano",
    tools=tools,
    input=input_list,
 )

def _serialize_output_item(item):
    """Convert SDK output objects into dictionaries we can send back to the API."""
    payload = json.loads(item.model_dump_json())
    payload.pop("status", None)
    return payload

input_list += [_serialize_output_item(item) for item in response.output]

for item in response.output:
    if item.type == "function_call" and item.name == "get_horoscope":
        args = item.arguments
        if isinstance(args, str):
            args = json.loads(args or "{}")
        elif args is None:
            args = {}
        horoscope = get_horoscope(**args)
        input_list.append({
            "type": "function_call_output",
            "call_id": item.call_id,
            "output": json.dumps(horoscope),
        })

print("Final input:")
print(json.dumps(input_list, indent=2))

response = client.responses.create(
    model="gpt-5-nano",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
 )

print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + (response.output_text or ""))

### Flow Recap

1. **Send** the user request along with tool schemas.
2. **Inspect** the assistant output—if it returns a `tool_call`, you own the execution.
3. **Respond** with a `tool_result` payload tied to the tool call ID.
4. **Read** the assistant’s final message that now includes the fresh data.

## Step 4: Wrap It in a Helper

Encapsulate the pattern so future notebooks or services can reuse it. The helper below runs the complete loop and returns both structured data and the assistant text.

In [ ]:
def _parse_arguments(raw_args):
    if isinstance(raw_args, str):
        raw_args = raw_args or "{}"
        return json.loads(raw_args)
    return raw_args or {}

def _serialize_output_item(item):
    payload = json.loads(item.model_dump_json())
    payload.pop("status", None)
    return payload

def run_assistant(prompt: str, model: str = "gpt-5-nano") -> dict:
    """Run the documented Responses API tool-calling loop and return structured data."""
    transcript = []
    input_list = [
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt}
            ],
        }
    ]

    initial = client.responses.create(
        model=model,
        tools=tools,
        input=input_list,
    )
    transcript.append(initial)
    input_list += [_serialize_output_item(item) for item in initial.output]

    call_outputs = []
    for item in initial.output:
        if item.type == "function_call":
            tool_fn = AVAILABLE_TOOLS.get(item.name)
            args = _parse_arguments(item.arguments)
            if not tool_fn:
                result_payload = {"success": False, "error": f"Unknown tool: {item.name}"}
            else:
                try:
                    result_payload = tool_fn(**args)
                except TypeError as exc:
                    result_payload = {"success": False, "error": str(exc)}
            serialized_result = json.dumps(result_payload)
            call_outputs.append({
                "call_id": item.call_id,
                "output": serialized_result,
            })
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": serialized_result,
            })

    final = client.responses.create(
        model=model,
        tools=tools,
        instructions="Summarize the tool findings clearly for the user.",
        input=input_list,
    )
    transcript.append(final)

    return {
        "prompt": prompt,
        "input_list": input_list,
        "tool_outputs": call_outputs,
        "assistant_text": final.output_text,
        "responses": transcript,
    }

demo = run_assistant("Please share today's Virgo horoscope.")
print(demo["assistant_text"])

## Optional: Async Version

Large agents often run in event loops. The async helper mirrors the same flow but awaits each network hop so you can orchestrate multiple assistants concurrently.

In [ ]:
from openai import AsyncOpenAI

async_client = AsyncOpenAI()

async def run_assistant_async(prompt: str, model: str = "gpt-5-nano") -> dict:
    """Async variant that mirrors the cumulative input workflow."""
    transcript = []
    input_list = [
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt}
            ],
        }
    ]

    initial = await async_client.responses.create(
        model=model,
        tools=tools,
        input=input_list,
    )
    transcript.append(initial)
    input_list += [_serialize_output_item(item) for item in initial.output]

    call_outputs = []
    for item in initial.output:
        if item.type == "function_call":
            tool_fn = AVAILABLE_TOOLS.get(item.name)
            if not tool_fn:
                continue
            args = _parse_arguments(item.arguments)
            try:
                result_payload = tool_fn(**args)
            except TypeError as exc:
                result_payload = {"success": False, "error": str(exc)}
            json_payload = json.dumps(result_payload)
            call_outputs.append({
                "call_id": item.call_id,
                "output": json_payload,
            })
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json_payload,
            })

    final = await async_client.responses.create(
        model=model,
        tools=tools,
        instructions="Summarize the tool findings clearly for the user.",
        input=input_list,
    )
    transcript.append(final)

    return {
        "prompt": prompt,
        "input_list": input_list,
        "tool_outputs": call_outputs,
        "assistant_text": final.output_text,
        "responses": transcript,
    }

async_demo = await run_assistant_async("Can you read my Capricorn horoscope?")
print(async_demo["assistant_text"])

### Built-In Responses API Tools at a Glance

- `code_interpreter`: Executes Python snippets in a managed sandbox and returns stdout, plots, and files for data wrangling workflows.
- `file_search`: Lets the assistant scan embeddings-indexed documents you have uploaded, returning citations or extracted passages.
- `function` (custom tools): The adapter we are using here—describe your own callable endpoints and the model will request them when it needs external data.
- `web_search` (beta regions): Allows the model to surface fresh public information, subject to your deployment and safety filters.
- `response.create` helpers: Chain multiple tool calls by continuing to send the growing transcript via the `input` list, just like the horoscope demo below.

#### Example: Using Built-In Web Search

Here's a minimal call that lets the model reach out for fresh information. The `web_search` tool requires no custom setup—just declare it, ask your question, and the Responses API handles the search call and citations automatically (ensure the feature is available in your region).

In [ ]:
web_search_demo = client.responses.create(
    model="gpt-4.1-mini",
    tools=[{"type": "web_search"}],
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "Find the latest news about breakthroughs in quantum computing and summarize one key development.",
                }
            ],
        }
    ],
)
print(web_search_demo.model_dump_json(indent=2))
print("\nSummary:")
print(web_search_demo.output_text or "(No text output)")

## ✅ Key Takeaways

✓ Tool calling bridges model creativity with your live systems.

✓ Built-in tools like `web_search`, `code_interpreter`, and `file_search` give you zero-setup superpowers—mix them with your custom functions for richer assistants.

✓ Always describe tools precisely with JSON Schema so the assistant knows when to use them.

✓ You must execute tool logic yourself—validate inputs, handle errors, and return structured output.

✓ Wrap the request → tool → response loop in helpers to keep production agents tidy.

—

**Great work!** You now have the blueprint for assistants that can reason, fetch, and act with confidence.